<a href="https://colab.research.google.com/github/BritCrit/monthly_automotive_incentives_autoblog/blob/main/car_incentives_autoblog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:

import pandas as pd

url = 'https://www.autoblog.com/new-car-incentives/'

def collect_incentives(url):
  return pd.read_html(url)

def find_page_count(df):
  page_count = df[0][-1:].iloc[0][0]
  return [int(s) for s in page_count.split() if s.isdigit()][0]\

def all_incenvtives(url):
  df_frames = []
  base_df = collect_incentives(url)
  df_frames.append(base_df[0])
  page_count = find_page_count(base_df)
  print(page_count)
  for n in range(2 , page_count):
    tmp_df = collect_incentives(f"https://www.autoblog.com/new-car-incentives/pg-{n}/")
    print(tmp_df)
    if tmp_df != None:
      df_frames.append(tmp_df[0])
    else: 
      pass
  return df_frames

def combine_dataframes(url):
  collected_dataframes = all_incenvtives(url)
  return pd.concat(collected_dataframes, ignore_index=True).ffill(axis=0)

master_df = combine_dataframes(url)

#Update columns 
master_df.columns = ['model', 'incentive', 'brand']
#Drop repeating lines and page breaks
master_df = master_df[master_df['incentive'] != master_df['brand']]
#Remove unicode formatting 
master_df.incentive = master_df.incentive.apply(lambda x: x.replace(u'\xa0', ' '))

master_df.to_csv("incentives.csv", index=False)

9
[                       Unnamed: 0 Manufacturer Financing Local Incentives
0                             BMW                    BMW              BMW
1                   2022 BMW 530e            up to $1000              NaN
2                    2022 BMW 540            up to $1000              NaN
3                    2022 BMW 740            up to $1000              NaN
4                   2022 BMW 745e            up to $1000              NaN
5                    2022 BMW 750            up to $3000              NaN
6                    2022 BMW 840            up to $1500              NaN
7         2022 BMW 840 Gran Coupe            up to $1000              NaN
8              2022 BMW ALPINA B7            up to $1000              NaN
9   2022 BMW ALPINA B8 Gran Coupe            up to $1000              NaN
10            2022 BMW ALPINA XB7            up to $1000              NaN
11       2022 BMW M235 Gran Coupe            up to $1000              NaN
12                  2022 BMW M240  

9
[                       Unnamed: 0 Manufacturer Financing Local Incentives
0                             BMW                    BMW              BMW
1                   2022 BMW 530e            up to $1000              NaN
2                    2022 BMW 540            up to $1000              NaN
3                    2022 BMW 740            up to $1000              NaN
4                   2022 BMW 745e            up to $1000              NaN
5                    2022 BMW 750            up to $3000              NaN
6                    2022 BMW 840            up to $1500              NaN
7         2022 BMW 840 Gran Coupe            up to $1000              NaN
8              2022 BMW ALPINA B7            up to $1000              NaN
9   2022 BMW ALPINA B8 Gran Coupe            up to $1000              NaN
10            2022 BMW ALPINA XB7            up to $1000              NaN
11       2022 BMW M235 Gran Coupe            up to $1000              NaN
12                  2022 BMW M240  

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,model,incentive,brand


In [ ]:
test = 


In [ ]:
test.unique()

array(['up to $750', 'up to $1000', 'from 1.99%',
       'up to $1000 / from 1.99%', 'up to $2000', 'up to $7500',
       'from 2.89%', 'up to $750 / from 1.99%', 'up to $3000',
       'up to $1500', 'from 2.99%', 'up to $2500', 'up to $500',
       'up to $2250', 'from 2.68%', 'up to $6970',
       'up to $1750 / from 0.0%', 'up to $550', 'up to $400',
       'up to $400 / from 1.9%', 'from 2.6%', 'up to $1000 / from 1.9%',
       'from 0.9%', 'up to $1250', 'up to $11500', 'up to $5000',
       'up to $2000 / from 1.9%', 'up to $250', 'up to $500 / from 0.9%',
       'from 2.39%', 'up to $500 / from 0.0%', 'up to $2000 / from 0.9%',
       'from 0.0%'], dtype=object)